In [2]:
import os
import re

def create_consolidated_file(root_directory, output_filename="consolidated_source.md"):
    """
    Crawls a directory for specified source files (.py, .md, .rst, .txt)
    and combines them into a single markdown file with a clickable index.

    Python code is enclosed in appropriate markdown code blocks.

    Args:
        root_directory (str): The path to the root folder to start searching from.
        output_filename (str): The name of the master markdown file to create.
    """
    # A list to hold the paths of all found source files.
    source_files = []

    # Define the file extensions to look for.
    included_extensions = ('.py', '.md', '.markdown', '.rst', '.txt')

    # Walk through the directory tree starting from the root_directory.
    for dirpath, _, filenames in os.walk(root_directory):
        for filename in filenames:
            # Check if the file has one of the desired extensions.
            if filename.lower().endswith(included_extensions):
                # Construct the full path and add it to our list.
                full_path = os.path.join(dirpath, filename)
                source_files.append(full_path)

    if not source_files:
        print(f"No relevant source files found in '{root_directory}'.")
        return

    # Sort the files alphabetically for a consistent order.
    source_files.sort()

    def create_anchor(path):
        """Creates a URL-friendly anchor from a file path."""
        relative_path = os.path.relpath(path, root_directory)
        # Replace problematic characters with a hyphen.
        anchor = re.sub(r'[\\/\s.]', '-', relative_path)
        return anchor.lower()

    try:
        # Open the output file in write mode with UTF-8 encoding.
        with open(output_filename, 'w', encoding='utf-8') as outfile:
            # --- 1. Write the Index ---
            outfile.write("# Consolidated Source Index\n\n")
            outfile.write("This document combines multiple source files. Below is the index of the included files.\n\n")

            for src_file in source_files:
                relative_path = os.path.relpath(src_file, root_directory)
                anchor = create_anchor(src_file)
                outfile.write(f"- [{relative_path}](#{anchor})\n")

            # --- 2. Write the Content ---
            for src_file in source_files:
                relative_path = os.path.relpath(src_file, root_directory)
                anchor = create_anchor(src_file)

                # Add a strong separator and a header for each file.
                outfile.write(f"\n\n---\n\n")
                outfile.write(f"## <a name=\"{anchor}\"></a>File: {relative_path}\n\n")

                try:
                    with open(src_file, 'r', encoding='utf-8') as infile:
                        content = infile.read()
                        
                        # For Python files, wrap the content in a python code block.
                        if src_file.lower().endswith('.py'):
                            outfile.write("```python\n")
                            outfile.write(content)
                            outfile.write("\n```")
                        # For other file types, write the content directly.
                        else:
                            outfile.write(content)

                except Exception as e:
                    outfile.write(f"*Error reading file: {e}*\n")

        print(f"Successfully created '{output_filename}' with {len(source_files)} files.")

    except IOError as e:
        print(f"Error writing to output file '{output_filename}': {e}")




In [3]:
if __name__ == '__main__':
    # --- Configuration ---
    # Set the path to the folder containing your source files.
    # Using '.' means the script will start in the same directory it is run from.
    SOURCE_DIRECTORY = r'C:\Users\basde\pyosmium\docs'

    # Set the name for the final combined markdown file.
    OUTPUT_FILE = 'pyosmiumdocs.md'

    # --- Run the script ---
    create_consolidated_file(SOURCE_DIRECTORY, OUTPUT_FILE)

Successfully created 'pyosmiumdocs.md' with 26 files.
